In [ ]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns

## 1- Método para estimación de datos faltantes

In [ ]:
data = pd.read_csv("./dataset.csv", header=0)


MISSING_VALUE = 999.99


def estimate_missing_values(data, k):

    # Separamos los datos en dos conjuntos:
    # - Completos: los datos que no tienen valores en MISSING_VALUE
    complete_df = data
    complete_df = complete_df[~complete_df.isin([MISSING_VALUE]).any(axis=1)]

    # - Incompletos: los datos que tienen al menos 1 valor en MISSING_VALUE
    incomplete_df = data
    incomplete_df = incomplete_df[incomplete_df.isin(
        [MISSING_VALUE]).any(axis=1)]

    # Estandarizamos ambos conjuntos
    scaler = StandardScaler()
    std_complete_df = pd.DataFrame(scaler.fit_transform(complete_df),columns = complete_df.columns)
    std_incomplete_df = pd.DataFrame(scaler.fit_transform(incomplete_df),columns = incomplete_df.columns)

    estimated_data = data.copy()      

    # Calculamos la distancia euclidea entre cada dato incompleto y todos los datos completos
    for i in range(len(incomplete_df)):
        incomplete_row_df = incomplete_df.iloc[[i]]
        std_incomplete_row = std_incomplete_df.iloc[[i]]
        
        missing_column_names = incomplete_row_df.columns[(incomplete_row_df == MISSING_VALUE).iloc[0]] 

        # Eliminamos los datos faltantes
        std_incomplete_row = std_incomplete_row.drop(missing_column_names, axis=1)

            # Eliminamos los datos faltantes
        std_complete_aux = std_complete_df.drop(missing_column_names, axis=1)
        distances = []
        for j in range(len(std_complete_aux)):
            # Almacenamos distancia para ordenamiento e índice de fila 
            distances.append((np.linalg.norm(std_incomplete_row - std_complete_aux.iloc[[j]]), j))

        # Calculamos los k vecinos más cercanos
        k_nearest_neighbors_indexes = map(lambda x: x[1], sorted(
            distances, key=lambda distance: distance[0])[:k])

        # Nos quedamos con los datos de los k vecinos más cercanos
        k_nearest_neighbors_df = complete_df.filter(
            items=k_nearest_neighbors_indexes, axis=0)

        # Calculamos la media de los k vecinos más cercanos para cada dato incompleto
        for missing_column_name in missing_column_names:
            estimated_data.at[incomplete_row_df.index[0], missing_column_name] = k_nearest_neighbors_df[missing_column_name].mean()
    
    return estimated_data

## 2- Comparación de datos sin estandarizar vs estandarizados mediante boxplots

In [ ]:
raw_data = data.drop("Sexo", axis=1)

trimmed_data = data[~data.isin([MISSING_VALUE]).any(axis=1)]
trimmed_data = trimmed_data.drop("Sexo", axis=1)

data.loc[data["Sexo"] == "F", "Sexo"] = 1
data.loc[data["Sexo"] == "M", "Sexo"] = -1

estimated = estimate_missing_values(data, k=5)

estimated_data = estimated.drop("Sexo", axis=1)

### 2.1- Comparación de estimadores

In [ ]:
print("DATOS SIN MODIFICAR")
display(raw_data.describe());

print("ELIMINANDO DATOS FALTANTES")
display(trimmed_data.describe());

print("RELLENANDO DATOS FALTANTES CON ESTIMACIÓN")
display(estimated_data.describe());

### 2.2- Comparación de datos no estandarizados vs estandarizados

In [ ]:
# *********************************************************************************************
# * Sin modificar datos
# *********************************************************************************************
# Plot de datos sin estandarizar
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].boxplot(raw_data)
axes[0].set_xticks([1, 2, 3], raw_data.columns)
axes[0].set_title("Boxplot de datos sin estandarizar")

# Plot de datos estandarizados
scaler = StandardScaler().fit(raw_data)
std_raw_data = pd.DataFrame(scaler.transform(raw_data), columns=raw_data.columns)
axes[1].boxplot(std_raw_data)
axes[1].set_xticks([1, 2, 3], raw_data.columns)
axes[1].set_title("Boxplot de datos estandarizados")


# *********************************************************************************************
# * Eliminando datos faltantes
# *********************************************************************************************

# Plot de datos sin estandarizar
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].boxplot(trimmed_data)
axes[0].set_xticks([1, 2, 3], trimmed_data.columns)
axes[0].set_title("Boxplot de datos sin estandarizar\n (Eliminando datos faltantes)")


# Plot de datos estandarizados
scaler = StandardScaler().fit(trimmed_data)
scaled_data = scaler.transform(trimmed_data)
axes[1].boxplot(pd.DataFrame(scaled_data))
axes[1].set_xticks([1, 2, 3], trimmed_data.columns)
axes[1].set_title("Boxplot de datos estandarizados\n (Eliminando datos faltantes)")

# *********************************************************************************************
# * Rellenando los datos faltantes con la media de los datos de los k vecinos más cercanos
# *********************************************************************************************

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].boxplot(estimated_data)
axes[0].set_xticks([1, 2, 3], estimated_data.columns)
axes[0].set_title("Boxplot de datos sin estandarizar\n (Rellenos con estimación)")


scaler = StandardScaler().fit(estimated_data)
scaled_data = scaler.transform(estimated_data)
# Plot de data escalada
axes[1].boxplot(pd.DataFrame(scaled_data))
axes[1].set_xticks([1, 2, 3], estimated_data.columns)
axes[1].set_title("Boxplot de datos estandarizados\n (Rellenos con estimación)")



### 2.3- Análisis de variables

In [ ]:

# *********************************************************************************************
# * Boxplots de cada variable por separado
# *********************************************************************************************

# Boxplots de variables sin modificar
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))
fig.suptitle("Boxplots de variables sin modificar")

axes[0].boxplot(raw_data["Calorías"])
axes[0].set_title("Calorías")

axes[1].boxplot(raw_data["Alcohol"])
axes[1].set_title("Alcohol")

axes[2].boxplot(raw_data["Grasas_sat"])
axes[2].set_title("Grasas_sat")

# Boxplots de variables eliminando muestras con datos faltantes
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))
fig.suptitle("Boxplots de variables eliminando muestras con datos faltantes")

axes[0].boxplot(trimmed_data["Calorías"])
axes[0].set_title("Calorías")

axes[1].boxplot(trimmed_data["Alcohol"])
axes[1].set_title("Alcohol")

axes[2].boxplot(trimmed_data["Grasas_sat"])
axes[2].set_title("Grasas_sat")

# Boxplots de variables rellenando datos faltantes
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))
fig.suptitle("Boxplots de variables rellenando datos faltantes")

axes[0].boxplot(estimated_data["Calorías"])
axes[0].set_title("Calorías")

axes[1].boxplot(estimated_data["Alcohol"])
axes[1].set_title("Alcohol")

axes[2].boxplot(estimated_data["Grasas_sat"])
axes[2].set_title("Grasas_sat")

## 3- Análisis de Grasas_sat, Alcohol, y Categorías en base al Sexo

In [ ]:
# Utilizando los datos sin estandarizar, y rellenando los datos faltantes
estimated_data.loc[estimated_data["Sexo"] == 1, "Sexo"] = "F"
estimated_data.loc[estimated_data["Sexo"] == -1, "Sexo"] = "M"
groupby_sex = estimated_data.groupby("Sexo", axis=0)

### 3.1- Estimadores

In [ ]:
grasas_sat_bysex = groupby_sex["Grasas_sat"]
alcohol_bysex = groupby_sex["Alcohol"]
calories_bysex = groupby_sex["Calorías"]

print("Grasas_sat")
display(grasas_sat_bysex.describe())

print("Alcohol")
display(alcohol_bysex.describe())

print("Calorías")
display(calories_bysex.describe())

### 3.2- Comparación con boxplots

In [ ]:
# *********************************************************************************************
# * Boxplots de Grasas_sat por Sexo
# *********************************************************************************************

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
fig.suptitle("Comparación de Grasas_sat entre sexo femenino y masculino")

axes[0].boxplot(groupby_sex.get_group("F")["Grasas_sat"])
axes[0].set_title("Femenino")

axes[1].boxplot(groupby_sex.get_group("M")["Grasas_sat"])
axes[1].set_title("Masculino")

# *********************************************************************************************
# * Boxplots de Alcohol por Sexo
# *********************************************************************************************

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
fig.suptitle("Comparación de Alcohol entre sexo femenino y masculino")

axes[0].boxplot(groupby_sex.get_group("F")["Alcohol"])
axes[0].set_title("Femenino")

axes[1].boxplot(groupby_sex.get_group("M")["Alcohol"])
axes[1].set_title("Masculino")

# *********************************************************************************************
# * Boxplots de Calorías por Sexo
# *********************************************************************************************

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
fig.suptitle("Comparación de Calorías entre sexo femenino y masculino")

axes[0].boxplot(groupby_sex.get_group("F")["Calorías"])
axes[0].set_title("Femenino")

axes[1].boxplot(groupby_sex.get_group("M")["Calorías"])
axes[1].set_title("Masculino")

## 4- Análisis de la variable Alcohol en base a las Calorías por categorías

In [ ]:
data = pd.read_csv("./dataset.csv", header=0)
data.loc[data["Sexo"] == "F", "Sexo"] = 1
data.loc[data["Sexo"] == "M", "Sexo"] = -1
estimated_data = estimate_missing_values(data, k=5)

calories_classification = estimated_data.copy()

CALORIES_CATEGORY = [{
    "floor": 0,
    "ceiling": 1100,
},
    {
    "floor": 1100,
    "ceiling": 1700,
},
    {
    "floor": 1700,
    "ceiling": -1,
}]

# Pisamos los datos de calorías con las cotas inferiores de las categorías
# Esto se realiza para aplicar ténicas de agrupación por categoría
for index, calories_category in enumerate(CALORIES_CATEGORY[:-1]):
    calories_classification.loc[(calories_classification["Calorías"] >= calories_category["floor"]) & (
        calories_classification["Calorías"] < calories_category["ceiling"]), 'Categoria de calorías'] = "CATE" + str(index) + ": " + str(calories_category["floor"]) + " - " + str(calories_category["ceiling"])

calories_classification.loc[calories_classification["Calorías"] >=
                           CALORIES_CATEGORY[-1]["floor"], 'Categoria de calorías'] = "CATE" + str(len(CALORIES_CATEGORY)) + ": > " + str(CALORIES_CATEGORY[-1]["floor"])


# Nos quedamos solo con las columnas de interés
df_alcohol = calories_classification.loc[:, ["Categoria de calorías", "Alcohol"]]
# Agrupamos por categoría de calorías
df_calories_groupby = df_alcohol.groupby("Categoria de calorías", axis=0)

# Armamos clasificación de alcohol por cada categoría
alcohol_classification = []
labels = []
for calories_group in df_calories_groupby.groups.keys():
    alcohol_classification.append(df_calories_groupby.get_group(
        calories_group)['Alcohol'].values)
    labels.append(calories_group)

fig, axes = plt.subplots(figsize=(10, 5))
axes.boxplot(alcohol_classification)
axes.set_xticks(range(1, len(labels) + 1), labels)
axes.set_title("Boxplots Alcohol por categoría de Calorías")



In [ ]:
# *********************************************************************************************
# Grafico de dispersión
fig, axes = plt.subplots(figsize=(10, 5))
axes.set_title("Grafico de dispersión de las categorías de Calorías")
sns.scatterplot(x="Calorías", y="Alcohol", data=calories_classification, ax=axes, hue="Categoria de calorías", palette="Set1")